# Cluster algorithm for customers value analysis in air company 

## 1. Task Description
Due to the limitation of the resources, customized strategies play a significant role in marketing. Besides, concentration of resources on customer with high value can maximize profit for the company. In this case, we need to design customized strategies for different type of customers.
### Goal
1. Build up a classifier for customers 
2. Analyze the features corresponding to different type of customers.
3. Compare the value of different type of customers
4. Design marketing strategies for different kind of customers

## 2. Dataset
The data set is in kaggle. You can download it from the link below.
https://www.kaggle.com/vinzzhang/aircompanycustomerinfo

## 3. Method
The whole step with
1. data preprocessing 
2. feature engineering including feature selection and normalization
3. Cluster
4. Analysis and Decision

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn.preprocessing
import sklearn.cluster

### 3.1 Data Preprocessing

In [2]:
### load data
path = 'air_data.csv'
df = pd.read_csv(path)
print(df.shape)

(62988, 44)


In [3]:
### quick view of the dataset
df.head(5)

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006/11/02,2008/12/24,男,6,.,北京,CN,31.0,2014/03/31,...,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007/02/19,2007/08/03,男,6,NaN,北京,CN,42.0,2014/03/31,...,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007/02/01,2007/08/30,男,6,.,北京,CN,40.0,2014/03/31,...,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008/08/22,2008/08/23,男,5,Los Angeles,CA,US,64.0,2014/03/31,...,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009/04/10,2009/04/15,男,6,贵阳,贵州,CN,48.0,2014/03/31,...,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39


In [4]:
df.dtypes

MEMBER_NO                    int64
FFP_DATE                    object
FIRST_FLIGHT_DATE           object
GENDER                      object
FFP_TIER                     int64
WORK_CITY                   object
WORK_PROVINCE               object
WORK_COUNTRY                object
AGE                        float64
LOAD_TIME                   object
FLIGHT_COUNT                 int64
BP_SUM                       int64
EP_SUM_YR_1                  int64
EP_SUM_YR_2                  int64
SUM_YR_1                   float64
SUM_YR_2                   float64
SEG_KM_SUM                   int64
WEIGHTED_SEG_KM            float64
LAST_FLIGHT_DATE            object
AVG_FLIGHT_COUNT           float64
AVG_BP_SUM                 float64
BEGIN_TO_FIRST               int64
LAST_TO_END                  int64
AVG_INTERVAL               float64
MAX_INTERVAL                 int64
ADD_POINTS_SUM_YR_1          int64
ADD_POINTS_SUM_YR_2          int64
EXCHANGE_COUNT               int64
avg_discount        

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
MEMBER_NO,62988.0,31494.500000,18183.213715,1.00,15747.750000,31494.500000,47241.250000,62988.000000
FFP_TIER,62988.0,4.102162,0.373856,4.00,4.000000,4.000000,4.000000,6.000000
AGE,62568.0,42.476346,9.885915,6.00,35.000000,41.000000,48.000000,110.000000
FLIGHT_COUNT,62988.0,11.839414,14.049471,2.00,3.000000,7.000000,15.000000,213.000000
BP_SUM,62988.0,10925.081254,16339.486151,0.00,2518.000000,5700.000000,12831.000000,505308.000000
EP_SUM_YR_1,62988.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
EP_SUM_YR_2,62988.0,265.689623,1645.702854,0.00,0.000000,0.000000,0.000000,74460.000000
SUM_YR_1,62437.0,5355.376064,8109.450147,0.00,1003.000000,2800.000000,6574.000000,239560.000000
SUM_YR_2,62850.0,5604.026014,8703.364247,0.00,780.000000,2773.000000,6845.750000,234188.000000
SEG_KM_SUM,62988.0,17123.878691,20960.844623,368.00,4747.000000,9994.000000,21271.250000,580717.000000


Check if there are duplication of member ID.

In [6]:
dup = df[df['MEMBER_NO'].duplicated()]
if len(dup) != 0:
    print('The duplications are as follows')
    print(dup)

Check if missing values are in dataset

In [7]:
df.isnull().any()

MEMBER_NO                  False
FFP_DATE                   False
FIRST_FLIGHT_DATE          False
GENDER                      True
FFP_TIER                   False
WORK_CITY                   True
WORK_PROVINCE               True
WORK_COUNTRY                True
AGE                         True
LOAD_TIME                  False
FLIGHT_COUNT               False
BP_SUM                     False
EP_SUM_YR_1                False
EP_SUM_YR_2                False
SUM_YR_1                    True
SUM_YR_2                    True
SEG_KM_SUM                 False
WEIGHTED_SEG_KM            False
LAST_FLIGHT_DATE           False
AVG_FLIGHT_COUNT           False
AVG_BP_SUM                 False
BEGIN_TO_FIRST             False
LAST_TO_END                False
AVG_INTERVAL               False
MAX_INTERVAL               False
ADD_POINTS_SUM_YR_1        False
ADD_POINTS_SUM_YR_2        False
EXCHANGE_COUNT             False
avg_discount               False
P1Y_Flight_Count           False
L1Y_Flight

We can find that SUM_YR_1 and SUM_YR_2, which means the consumption in $i$ year, have missing values. There are two potential reason: the consumption is 0 in the $i^{th}$ year or it wasn't recorded during collection. In this case, we can drop those kind of data. 

In [9]:
df = df[df['SUM_YR_1'].notnull() & df['SUM_YR_2'].notnull()]
df = df[(df['SUM_YR_1'] != 0) | (df['SUM_YR_2'] != 0)]
print(df.shape)

(62044, 44)


### 3.2 Feature Engineering

#### 3.2.1 RMF Model
The RMF model is a classic model for customer value analysis. The meaning of terms are as follows:

1. Recency: the interval of recent consumption

2. Frequency: the consumption frequency

3. Monetary Value: the total amount of consumption

In RFM model, the customers would be seperated into 5 classes including important retention customer, important development customer, important retaining customer, general customer, customer with low value.

#### 3.2.2 LRFMC Model
LRFMC model is an extension of RMF model. The features in this model are:
1. Length of Relationship: reflect the active length
2. Recency: the time interval of recent consumption, reflect the active state
3. Frequency: the consumption frequency, reflect the loyalty of the customer
4. Mileage: the total amount of flight, reflect the dependence of the company
5. Coefficient of Discount: the average discount of the customer, reflect the value of a customer

We can derive the features from the dataset. The corresponding string would be:
1. L = LOAD_TIME - FFP_DATE
2. R = LAST_TO_END
3. F = FLIGHT_COUNT
4. M = SEG_KM_SUM
5. C = avg_discount

Now let's generate a new table

In [10]:
load_time = datetime.datetime.strptime('2014/03/31','%Y/%m/%d')
ffp_dates = [datetime.datetime.strptime(ffp_date,'%Y/%m/%d') for ffp_date in df['FFP_DATE']]
df['LEN_REL'] = [(load_time-ffp_date).days for ffp_date in ffp_dates]

Extract the related features from the dataset

In [12]:
Features = ['LEN_REL','FLIGHT_COUNT','avg_discount','SEG_KM_SUM','LAST_TO_END']
data = df[Features]
features = ['L','F','C','M','R']
data.columns = features
print(data.head(5))
data.describe().T

      L    F         C       M   R
0  2706  210  0.961639  580717   1
1  2597  140  1.252314  293678   7
2  2615  135  1.254676  283712  11
3  2047   23  1.090870  281336  97
4  1816  152  0.970658  309928   5


,count,mean,std,min,25%,50%,75%,max
L,62044.0,1488.691090,847.880920,365.000000,735.000000,1278.000000,2182.000000,3437.0
F,62044.0,11.971359,14.110619,2.000000,3.000000,7.000000,15.000000,213.0
C,62044.0,0.722180,0.184833,0.136017,0.613085,0.712162,0.809293,1.5
M,62044.0,17321.694749,21052.728111,368.000000,4874.000000,10200.000000,21522.500000,580717.0
R,62044.0,172.532703,181.526164,1.000000,29.000000,105.000000,260.000000,731.0


### Normalization
Since the scope of each feature are different, we need to normalize the data to avoid the gap.

In [14]:
### Two ways
#data = (data-data.mean(axis=0))/(data.std(axis=0))
sp = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
data = sp.fit_transform(data)
data = pd.DataFrame(data, columns = features)
data_dp = data.copy()
data.describe().T

,count,mean,std,min,25%,50%,75%,max
L,62044.0,-6.707416e-16,1.000008,-1.325304,-0.888919,-0.248493,0.817703,2.297875
F,62044.0,-8.880299e-14,1.000008,-0.706662,-0.635793,-0.352316,0.214637,14.246736
C,62044.0,-2.380935e-15,1.000008,-3.171335,-0.590238,-0.054200,0.471308,4.208258
M,62044.0,-1.160847e-15,1.000008,-0.805303,-0.591268,-0.338282,0.199539,26.761370
R,62044.0,-7.294065e-15,1.000008,-0.944955,-0.790706,-0.372030,0.481848,3.076536


### 3.3 Model Training and Prediction
What we want to do is to cluster the customer into 5 groups. We'll apply k-means to cluster the data

In [15]:
km = sklearn.cluster.KMeans(n_clusters = 5, n_jobs = 4)
km.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=4, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [16]:
r1 = pd.Series(km.labels_).value_counts()
r2 = pd.DataFrame(km.cluster_centers_)
r = pd.concat([r2,r1],axis=1)
r.columns = list(data.columns)+['counts']
r

,L,F,C,M,R,counts
0,1.160640,-0.086936,-0.155829,-0.094858,-0.377234,15741
1,-0.700235,-0.161157,-0.255075,-0.160977,-0.414876,24661
2,-0.313575,-0.573978,-0.173070,-0.536807,1.686238,12125
3,0.051892,-0.226668,2.192257,-0.231054,-0.003246,4181
4,0.483332,2.483222,0.308633,2.424743,-0.799390,5336


### 4.5 Analysis and Decision

### Reference
https://lab.datafountain.cn/forum?id=67&tab=first